In [14]:
import requests
import httpx 

In [24]:
path = "/api/events/10"
base_url = "http://localhost:8002"
endpoint = f"{base_url}{path}"
endpoint

'http://localhost:8002/api/events/10'

In [26]:
response = requests.get(endpoint)
data = response.json()
print(data)

{'idd': 10}
